In [1]:
import os
import time
import numpy as np
from ase import Atoms
from ase import neighborlist
from ase.io import write

In [2]:
'''
site energy -> activation energy -> diffusion rate

site energy(E_i) = -N*(E_b)/2

activation energy(E_a) = E_0 + alpha * E_r
E_r = E_i(end) - E_i(start)

diffusion rate = f*exp(-E_a/(k*T)) (Arrhenius)
f is ~ 10^13 for most metals
'''

def get_site_energy(bond_energy, bond_num):
    return -bond_num * bond_energy/2

def get_activation_energy(e_start, e_end, alpha = 0.1, e0 = 0):
    e_reaction = e_end-e_start
    if e_reaction>=0:
        return e0 + (1+alpha)*e_reaction
    else:
        return e0 + alpha*e_reaction

def get_diffusion_rate(e_a, T=300, f=1E13):
    k_B = 8.617333262145e-5  # Boltzmann constant in eV/K
    return f*np.exp(-e_a/(k_B*T))

In [3]:
# function to create fcc lattice
def create_fcc_lattice(symbol, lattice_constant, repetitions):
    """
    Create an FCC lattice of atoms.

    Args:
    - symbol: The chemical symbol of the atoms.
    - lattice_constant: The lattice constant of the FCC lattice.
    - repetitions: The number of repetitions of the unit cell in each direction.

    Returns:
    - An Atoms object representing the FCC lattice.
    """

    # Define the positions of the atoms in the unit cell
    positions = [[0, 0, 0],
                 [0.5, 0.5, 0],
                 [0.5, 0, 0.5],
                 [0, 0.5, 0.5]]

    # Scale the positions by the lattice constant
    positions = [[x * lattice_constant for x in pos] for pos in positions]

    # Create the FCC lattice
    fcc_lattice = Atoms([symbol] * 4, positions=positions, cell=[lattice_constant] * 3, pbc=True)

    # Repeat the unit cell in all directions
    fcc_lattice *= repetitions

    return fcc_lattice


In [4]:
# test
test_lattice = create_fcc_lattice('Cu', 3.6, [10, 10, 3])
write('test_lattice.xyz', test_lattice)


/tmp/ipykernel_30705/1028488303.py:28: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  fcc_lattice *= repetitions
